In [1]:
import torch
import evaluate
import numpy as np
from datasets import load_from_disk
from transformers import Trainer, TrainingArguments, ViTForImageClassification

/home/calvin/Projects/icon-classifier/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset

In [2]:
dataset = load_from_disk("../data/processed/huggingface")
labels = dataset["train"].features["label"].names

# Training

In [3]:
accuracy = evaluate.load("accuracy")

def compute_accuracy(p):
    pred = np.argmax(p.predictions, axis=1)
    lab = p.label_ids
    return accuracy.compute(predictions=pred, references=lab)

In [4]:
# Load the pre-trained model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,  # Ignore the classifier's size mismatch
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="../models/results",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="../models/logs",
    logging_steps=10,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_accuracy
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([63]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([63, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/calvin/Projects/icon-classifier/.venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
# Train the model
trainer.train()

  3%|▎         | 10/342 [04:53<2:40:12, 28.95s/it]

{'loss': 4.1874, 'grad_norm': 8.067720413208008, 'learning_rate': 4.853801169590643e-05, 'epoch': 0.09}


  6%|▌         | 20/342 [09:42<2:33:25, 28.59s/it]

{'loss': 3.9065, 'grad_norm': 8.660638809204102, 'learning_rate': 4.707602339181287e-05, 'epoch': 0.18}


  9%|▉         | 30/342 [13:06<1:45:28, 20.28s/it]

...

 99%|█████████▉| 340/342 [2:16:14<00:40, 20.14s/it]

{'loss': 0.0166, 'grad_norm': 0.20978698134422302, 'learning_rate': 2.9239766081871344e-07, 'epoch': 2.98}


100%|██████████| 342/342 [2:16:45<00:00, 23.99s/it]

{'train_runtime': 8205.3428, 'train_samples_per_second': 0.663, 'train_steps_per_second': 0.042, 'train_loss': 0.8064742495975735, 'epoch': 3.0}


TrainOutput(global_step=342, training_loss=0.8064742495975735, metrics={'train_runtime': 8205.3428, 'train_samples_per_second': 0.663, 'train_steps_per_second': 0.042, 'total_flos': 4.219419671059784e+17, 'train_loss': 0.8064742495975735, 'epoch': 3.0})

In [6]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate()
trainer.log_metrics("eval", eval_results)
trainer.save_metrics("eval", eval_results)

100%|██████████| 29/29 [03:18<00:00,  6.86s/it]

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =        1.0
  eval_loss               =      0.026
  eval_runtime            = 0:03:27.34
  eval_samples_per_second =       2.19
  eval_steps_per_second   =       0.14


In [7]:
# Save the final trained model
model.save_pretrained("../models/results/final_model")

# Inference

In [ ]:
model = ViTForImageClassification.from_pretrained("./final_model")
model.eval()

In [ ]:
# Pick a training image to test with
example_path = "../data/processed/selected/480px-ISO_7000_-_Ref-No_0082.svg.png"
expected_class = "0082"
expected_class_idx = labels.index(expected_class)
example = load_image({"image": example_path}, t)
example = example["pixel_values"].unsqueeze(0)  # Add batch dimension

In [ ]:
example

In [ ]:
# Perform inference
with torch.no_grad():  # Disable gradient computation during inference
    outputs = model(example)
    logits = outputs.logits

# Convert logits to probabilities and get predicted class
probs = torch.softmax(logits, dim=-1)
predicted_class_idx = torch.argmax(probs, dim=-1).item()

In [ ]:
predicted_class_idx

In [ ]:
expected_class_idx

In [ ]:
probs